In [ ]:
################################################################################
#   Copyright (c) 2022 LASSE
#
#   This program is free software; you can redistribute it and/or modify
#   it under the terms of the GNU General Public License version 3 as
#   published by the Free Software Foundation;
#
#   This program is distributed in the hope that it will be useful,
#   but WITHOUT ANY WARRANTY; without even the implied warranty of
#   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#   GNU General Public License for more details.
#
#   You should have received a copy of the GNU General Public License
#   along with this program; if not, write to the Free Software
#   Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA  02111-1307  USA
################################################################################
# Based on Public Domain code written by LASSE: Ailton Oliveira, Aldebaro Klautau, Arthur Nascimento, Diego Gomes, Jamelly Ferreira, João Borges, Luan Gonçalves, and Walter Frazão.

"""Trains a deep NN for choosing top-K beams
Adapted by AK: Aug 7, 2018
See
https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/
and
https://stackoverflow.com/questions/45642077/do-i-need-to-use-one-hot-encoding-if-my-output-variable-is-binary
See for explanation about convnet and filters:
https://datascience.stackexchange.com/questions/16463/what-is-are-the-default-filters-used-by-keras-convolution2d
and
http://cs231n.github.io/convolutional-networks/
"""

import csv
import numpy as np
import argparse
import copy
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import model_from_json, Model, Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, Conv2D, add,\
    Flatten, MaxPooling2D, Dense, Reshape, Input, Dropout, concatenate, Softmax, ReLU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam
from keras.metrics import top_k_categorical_accuracy
from sklearn.model_selection import train_test_split
from scipy.special import softmax

In [ ]:
class ModelHandler:
    
    
    def createArchitecture(self,model_type,num_classes,input_shape,chain):
        '''
        Returns a NN model.
        modelType: a string which defines the structure of the model
        numClasses: a scalar which denotes the number of classes to be predicted
        input_shape: a tuple with the dimensions of the input of the model
        chain: a string which indicates if must be returned the complete model 
        up to prediction layer, or a segment of the model.
        '''
        
        if(model_type == 'inception_single'):
            input_inc = Input(shape = input_shape)

            tower_1 = Conv2D(4, (1,1), padding='same', activation='relu')(input_inc)
            tower_1 = Conv2D(8, (2,2), padding='same', activation='relu')(tower_1)
            tower_1 = Conv2D(16, (3,3), padding='same', activation='relu')(tower_1)
            tower_2 = Conv2D(4, (1,1), padding='same', activation='relu')(input_inc)
            tower_2 = Conv2D(16, (3,3), padding='same', activation='relu')(tower_2)
            tower_2 = Conv2D(16, (5,5), padding='same', activation='relu')(tower_2)
            tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_inc)
            tower_3 = Conv2D(4, (1,1), padding='same', activation='relu')(tower_3)
            
            output = concatenate([tower_1, tower_2, tower_3], axis = 3)
            
            if(chain=='segment'):
                architecture = output
                
            else:
                output = Dropout(0.25)(output)
                output = Flatten()(output)
                out = Dense(num_classes,activation='softmax')(output)
                
                architecture = Model(inputs = input_inc, outputs = out)

        elif(model_type == 'light_image'):
            nTx=32
            nRx=8
            batchsize=64
            splits=[0.8,0.2,0.1]
            posMatShape=[100,700,1]
            nClasses=num_classes
            nresblocks=3
            nChannels=32
            dropout_rate=0.5

            conv2D_1 = Conv2D(nChannels, (7, 7), padding="same", activation="relu")
            conv2D_2 = Conv2D(nChannels, (3, 3), padding="same", activation="relu")
            resBlocks = Sequential()
            resBlocks.add(
                self.build_resblock(
                    input_shape=(*input_shape[:-1], nChannels), nChannels=nChannels)
            )
            for i in range(nresblocks):
                resBlocks.add(
                self.build_resblock(
                    input_shape=(resBlocks.layers[-1].output_shape[1:]),
                    nChannels=nChannels,
                ))
            dense_1 = Dense(2 * nClasses, activation="relu")
            dense_2 = Dense(nClasses, activation="relu")
            dropout = Dropout(dropout_rate)
            flatten = Flatten()
            softmax = Softmax()

            input_inc= Input(shape=input_shape)

            x = conv2D_1(input_inc)
            x = conv2D_2(x)
            x = resBlocks(x)
            x = flatten(x)
            x = dense_1(x)
            # if training:
            #     x = self.dropout(x, training=training)
            x = dropout(x)
            x = dense_2(x)
            x = softmax(x)
            architecture = Model(inputs = input_inc, outputs = x)

        
        elif(model_type == 'coord_mlp'):
            input_coord = Input(shape = input_shape)
            
            layer = Dense(4,activation='relu')(input_coord)
            layer = Dense(16,activation='relu')(layer)
            layer = Dense(64,activation='relu')(layer)
            out = Dense(num_classes,activation='softmax')(layer)
            
            architecture = Model(inputs = input_coord, outputs = out)
            
        elif(model_type == 'lidar_marcus'):
            dropProb=0.3
            input_lid = Input(shape = input_shape)
                        
            layer = Conv2D(10,kernel_size=(13,13),
                                activation='relu',
                                padding="SAME",
                                input_shape=input_shape)(input_lid)
            layer = Conv2D(30, (11, 11), padding="SAME", activation='relu')(layer)
            layer = Conv2D(25, (9, 9), padding="SAME", activation='relu')(layer)
            layer = MaxPooling2D(pool_size=(2, 1))(layer)
            layer = Dropout(dropProb)(layer)
            layer = Conv2D(20, (7, 7), padding="SAME", activation='relu')(layer)
            layer = MaxPooling2D(pool_size=(1, 2))(layer)
            layer = Conv2D(15, (5, 5), padding="SAME", activation='relu')(layer)
            layer = Dropout(dropProb)(layer)
            layer = Conv2D(10, (3, 3), padding="SAME", activation='relu')(layer)
            layer = Conv2D(1, (1, 1), padding="SAME", activation='relu')(layer)
            layer = Flatten()(layer)
            out = Dense(num_classes,activation='softmax')(layer)
            
            architecture = Model(inputs = input_lid, outputs = out)
            
        elif(model_type == 'lidar_simple'):
            dropProb=0.3
            input_lid = Input(shape = input_shape)
                        
            layer = Conv2D(10,kernel_size=(13,13),
                                activation='relu',
                                padding="SAME",
                                input_shape=input_shape)(input_lid)
            layer = Conv2D(10, (11, 11), padding="SAME", activation='relu')(layer)
            layer = MaxPooling2D(pool_size=(3, 5))(layer)
            layer = Dropout(dropProb)(layer)
            layer = Conv2D(10, (7, 7), padding="SAME", activation='relu')(layer)
            layer = MaxPooling2D(pool_size=(1, 2))(layer)
            layer = Dropout(dropProb)(layer)
            layer = Conv2D(10, (3, 3), padding="SAME", activation='relu')(layer)
            layer = MaxPooling2D(pool_size=(1, 2))(layer)
            layer = Dropout(dropProb)(layer)
            layer = Flatten()(layer)
            out = Dense(num_classes,activation='softmax')(layer)
            
            architecture = Model(inputs = input_lid, outputs = out)
                        
        
        return architecture

    def build_resblock(self, input_shape, nChannels, maxpooling=True):
        input = Input(shape=input_shape)
        x = Conv2D(nChannels, (3, 3), padding="same", activation="relu")(input)
        x = Conv2D(nChannels, (3, 3), padding="same", activation=None)(x)
        x = input + x
        x = ReLU()(x)
        if maxpooling:
            x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(x)
        return Model(inputs=input, outputs=x)


In [ ]:
###############################################################################
# Support functions
###############################################################################

# For description about top-k, including the explanation on how they treat ties (which can be misleading
# if your classifier is outputting a lot of ties (e.g. all 0's will lead to high top-k)
# https://www.tensorflow.org/api_docs/python/tf/nn/in_top_k


def top_10_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=10)


def top_30_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=30)


def top_50_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=50)


def top_100_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=100)


def sub2ind(array_shape, rows, cols):
    ind = rows * array_shape[1] + cols
    ind[ind < 0] = -1
    ind[ind >= array_shape[0] * array_shape[1]] = -1
    return ind


def ind2sub(array_shape, ind):
    ind[ind < 0] = -1
    ind[ind >= array_shape[0] * array_shape[1]] = -1
    rows = ind.astype("int") / array_shape[1]
    cols = ind % array_shape[1]
    return (rows, cols)


def beamsLogScale(y, thresholdBelowMax):
    y_shape = y.shape

    for i in range(0, y_shape[0]):
        thisOutputs = y[i, :]
        logOut = 20 * np.log10(thisOutputs + 1e-30)
        minValue = np.amax(logOut) - thresholdBelowMax
        zeroedValueIndices = logOut < minValue
        thisOutputs[zeroedValueIndices] = 0
        thisOutputs = thisOutputs / sum(thisOutputs)
        y[i, :] = thisOutputs

    return y


def getBeamOutput(output_file):

    thresholdBelowMax = 60

    print("Reading dataset...", output_file)
    output_cache_file = np.load(output_file)
    yMatrix = output_cache_file["output_classification"]

    num_classes = 256

    pairs = [[i, j] for i in range(8) for j in range(32)]
    yMatrix = yMatrix.tolist()
    indexes = [pairs.index(yMatrix[item]) for item in range(len(yMatrix))]

    y = to_categorical(indexes, num_classes)

    # yMatrix = np.abs(yMatrix)

    # yMatrix /= np.max(yMatrix)
    # yMatrixShape = yMatrix.shape
    # num_classes = yMatrix.shape[1] * yMatrix.shape[2]

    # y = yMatrix.reshape(yMatrix.shape[0],num_classes)
    # y = beamsLogScale(y,thresholdBelowMax)

    return y, num_classes

In [ ]:
###############################################################################
# Data configuration
###############################################################################
tf.device("/device:GPU:0")

coord = False
img = False
lidar = True

num_epochs = 2
batch_size = 32
tgtRec = 3
seed = 0

pathToModel = (
    "./models/" + str(num_epochs) + "epochs"
)

np.random.seed(seed)

if coord == True:
    # coord_train_input_file = "/mnt/DATA/codes/mimo-python/simple_street2_part1_60_nTx32_nRx8_coordinates_input.npz"
    coord_train_input_file = (
        "/mnt/DATA/codes/mimo-python/simple_street60_nTx32_nRx8_coordinates_input.npz"
    )
    coord_train_cache_file = np.load(coord_train_input_file)
    X_coord = coord_train_cache_file["input_classification"]
    X_coord_train, X_coord_validation = train_test_split(
        X_coord, test_size=0.2, random_state=seed, shuffle=True
    )

    coord_train_input_shape = X_coord_train.shape

if img == True:
    nCh = 1  # The number of channels of the image
    
    # img_train_input_file = "/mnt/DATA/codes/mimo-python/simple_street2_part1_60_nTx32_nRx8_positionMatrix_input.npz"
    img_train_input_file = "/mnt/DATA/codes/mimo-python/simple_street60_nTx32_nRx8_positionMatrix_input.npz"
    img_train_cache_file = np.load(img_train_input_file)
    X_img = img_train_cache_file["input_classification"]
    # X_img_train, X_img_validation = train_test_split(X_img, test_size=0.2, random_state=seed, shuffle=True)
    print("Reading dataset... ", img_train_input_file)

    # img_train_input_shape = X_img_train.shape

if lidar == True:
    lidar_train_input_file = ""
    lidar_train_cache_file = np.load(lidar_train_input_file)
    X_lidar = lidar_train_cache_file["input"]
    X_lidar_train, X_lidar_validation = train_test_split(
        X_lidar, test_size=0.2, random_state=seed, shuffle=True
    )
    print("Reading dataset... ", lidar_train_input_file)
    lidar_train_input_shape = X_lidar_train.shape

In [ ]:
###############################################################################
# Output configuration
# output_file = '/mnt/DATA/codes/mimo-python/simple_street2_part1_60_nTx32_nRx8_beams_output.npz'
output_file = "/mnt/DATA/codes/mimo-python/simple_street60_nTx32_nRx8_beams_output.npz"
y_output, num_classes = getBeamOutput(output_file)
# y_train, y_validation = train_test_split(y_output, test_size=0.2, random_state=seed, shuffle=True)

indices = np.arange(len(y_output))

(
    X_img_train,
    X_img_validation,
    y_train,
    y_validation,
    indices_train,
    indices_validation,
) = train_test_split(
    X_img, y_output, indices, test_size=0.2, random_state=seed, shuffle=True
)

np.savez(
    "./indices", indices_train=indices_train, indices_validation=indices_validation
)

img_train_input_shape = X_img_train.shape

In [ ]:
##############################################################################
# Model configuration
##############################################################################

# multimodal
multimodal = [coord, img, lidar]
plot = True  # Active Plot output

modelHand = ModelHandler()
opt = Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam",
)

if coord:
    coord_model = modelHand.createArchitecture(
        "coord_mlp", num_classes, coord_train_input_shape[1], "complete"
    )
if img:
    if nCh == 1:
        img_model = modelHand.createArchitecture(
            "light_image",
            num_classes,
            [img_train_input_shape[1], img_train_input_shape[2], 1],
            "complete",
        )
    else:
        img_model = modelHand.createArchitecture(
            "light_image",
            num_classes,
            [
                img_train_input_shape[1],
                img_train_input_shape[2],
                img_train_input_shape[3],
            ],
            "complete",
        )
if lidar:
    lidar_model = modelHand.createArchitecture(
        "lidar_marcus",
        num_classes,
        [
            lidar_train_input_shape[1],
            lidar_train_input_shape[2],
            lidar_train_input_shape[3],
        ],
        "complete",
    )

if sum(multimodal) == 2:
    if coord and lidar:
        combined_model = concatenate([coord_model.output, lidar_model.output])
        z = Dense(num_classes, activation="relu")(combined_model)
        model = Model(inputs=[coord_model.input, lidar_model.input], outputs=z)
        model.compile(
            loss=categorical_crossentropy,
            optimizer=opt,
            metrics=[
                metrics.categorical_accuracy,
                metrics.top_k_categorical_accuracy,
                top_10_accuracy,
                top_30_accuracy,
                top_50_accuracy,
                top_100_accuracy,
            ],
        )
        model.summary()
        hist = model.fit(
            [X_coord_train, X_lidar_train],
            y_train,
            validation_data=([X_coord_validation, X_lidar_validation], y_validation),
            epochs=num_epochs,
            batch_size=batch_size,
        )

    elif coord and img:
        combined_model = concatenate([coord_model.output, img_model.output])
        z = Dense(num_classes, activation="relu")(combined_model)
        model = Model(inputs=[coord_model.input, img_model.input], outputs=z)
        model.compile(
            loss=categorical_crossentropy,
            optimizer=opt,
            metrics=[
                metrics.categorical_accuracy,
                metrics.top_k_categorical_accuracy,
                top_10_accuracy,
                top_30_accuracy,
                top_50_accuracy,
                top_100_accuracy,
            ],
        )
        model.summary()
        hist = model.fit(
            [X_coord_train, X_img_train],
            y_train,
            validation_data=([X_coord_validation, X_img_validation], y_validation),
            epochs=num_epochs,
            batch_size=batch_size,
        )
        model.save(pathToModel)

    else:
        combined_model = concatenate([lidar_model.output, img_model.output])
        z = Dense(num_classes, activation="relu")(combined_model)
        model = Model(inputs=[lidar_model.input, img_model.input], outputs=z)
        model.compile(
            loss=categorical_crossentropy,
            optimizer=opt,
            metrics=[
                metrics.categorical_accuracy,
                metrics.top_k_categorical_accuracy,
                top_10_accuracy,
                top_30_accuracy,
                top_50_accuracy,
                top_100_accuracy,
            ],
        )
        model.summary()
        hist = model.fit(
            [X_lidar_train, X_img_train],
            y_train,
            validation_data=([X_lidar_validation, X_img_validation], y_validation),
            epochs=num_epochs,
            batch_size=batch_size,
        )
elif sum(multimodal) == 3:
    combined_model = concatenate(
        [lidar_model.output, img_model.output, coord_model.output]
    )
    z = Dense(num_classes, activation="relu")(combined_model)
    model = Model(
        inputs=[lidar_model.input, img_model.input, coord_model.input], outputs=z
    )
    model.compile(
        loss=categorical_crossentropy,
        optimizer=opt,
        metrics=[
            metrics.categorical_accuracy,
            metrics.top_k_categorical_accuracy,
            top_10_accuracy,
            top_30_accuracy,
            top_50_accuracy,
            top_100_accuracy,
        ],
    )
    model.summary()
    hist = model.fit(
        [X_lidar_train, X_img_train, X_coord_train],
        y_train,
        validation_data=(
            [X_lidar_validation, X_img_validation, X_coord_validation],
            y_validation,
        ),
        epochs=num_epochs,
        batch_size=batch_size,
    )

else:
    if coord:
        model = coord_model
        model.compile(
            loss=categorical_crossentropy,
            optimizer=opt,
            metrics=[
                metrics.categorical_accuracy,
                metrics.top_k_categorical_accuracy,
                top_10_accuracy,
                top_30_accuracy,
                top_50_accuracy,
                top_100_accuracy,
            ],
        )
        model.summary()
        hist = model.fit(
            X_coord_train,
            y_train,
            validation_data=(X_coord_validation, y_validation),
            epochs=num_epochs,
            batch_size=batch_size,
        )

    elif img:
        model = img_model
        model.compile(
            loss=categorical_crossentropy,
            optimizer=opt,
            metrics=[
                metrics.categorical_accuracy,
                metrics.top_k_categorical_accuracy,
                top_10_accuracy,
                top_30_accuracy,
                top_50_accuracy,
                top_100_accuracy,
            ],
        )
        model.summary()
        hist = model.fit(
            X_img_train,
            y_train,
            validation_data=(X_img_validation, y_validation),
            epochs=num_epochs,
            batch_size=batch_size,
        )
        model.save(pathToModel)

    else:
        model = lidar_model
        model.compile(
            loss=categorical_crossentropy,
            optimizer=opt,
            metrics=[
                metrics.categorical_accuracy,
                metrics.top_k_categorical_accuracy,
                top_10_accuracy,
                top_30_accuracy,
                top_50_accuracy,
                top_100_accuracy,
            ],
        )
        model.summary()
        hist = model.fit(
            X_lidar_train,
            y_train,
            epochs=num_epochs,
            batch_size=batch_size,
            validation_data=(X_lidar_validation, y_validation),
        )

with open("history.txt", "w") as f:
    f.write(str(hist.history))

"""##Baselines cores"""

beam_weights = {}
for i in range(y_train.shape[1]):
    beam_weights[i] = 0

for i in range(y_train.shape[0]):
    scene_array = y_train[i, :]
    beam_weights[np.argmax(scene_array)] += 1

ocurrence = np.zeros((1, y_validation.shape[1]))
oc_factor = sum(beam_weights.values())
for b_index in beam_weights.keys():
    ocurrence[0, b_index] = beam_weights[b_index] / oc_factor

# ocurrence
ocurrence_input = np.repeat(ocurrence, y_validation.shape[0], axis=0)
ocurrence_output = softmax(ocurrence_input, axis=1)
# rand
rand_input = np.random.rand(X_img_validation.shape[0], 256)
rand_output = softmax(rand_input, axis=1)

"""#Accuracy / epochs"""

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(
    hist.history["categorical_accuracy"], "b", label="Categorical accuracy", linewidth=2
)
ax.plot(hist.history["top_k_categorical_accuracy"], "k--", label="Top 5", linewidth=2)
ax.plot(hist.history["top_10_accuracy"], "m", label="Top 10", linewidth=2)
ax.plot(hist.history["top_30_accuracy"], "g--", label="Top 30", linewidth=2)
ax.plot(hist.history["top_50_accuracy"], "r", label="Top 50", linewidth=2)
ax.set(xlabel="Epochs", ylabel="Accuracy")
ax.grid()
plt.legend()
# plt.title('LIDAR data')
plt.savefig("a.png")

"""##Creating Baselines"""



predict_top_k = {}
ocurrence_top_k = {}
random_top_k = {}
y_predict = model.predict(X_img_validation)
for i in range(1, 51):
    predict_top_k[f"top_{i}"] = (
        np.sum(top_k_categorical_accuracy(y_validation, y_predict, k=i))
        / y_predict.shape[0]
    )
    ocurrence_top_k[f"top_{i}"] = (
        np.sum(top_k_categorical_accuracy(y_validation, ocurrence_output, k=i))
        / y_predict.shape[0]
    )
    random_top_k[f"top_{i}"] = (
        np.sum(top_k_categorical_accuracy(y_validation, rand_output, k=i))
        / y_predict.shape[0]
    )

"""## TOP 10 acurracy"""

plt.figure(figsize=(7, 4))
plt.hist(predict_top_k["top_10"])
plt.hist(ocurrence_top_k["top_10"])
plt.hist(random_top_k["top_10"])

plt.figure(figsize=(7, 4))
plt.xlabel("Top k")
plt.ylabel("Accuracy")
plt.plot(list(predict_top_k.values()), label="NN")
plt.plot(list(ocurrence_top_k.values()), label="Ocurrence")
plt.plot(list(random_top_k.values()), label="Dummy")
plt.grid()
plt.legend()

"""# Save output"""

fileNameIdentifier = (
    "./output/"
    + str(num_epochs)
    + "epochs_output"
)
f = open(fileNameIdentifier + ".txt", "w")
f.write(str(hist.history))
f.close()